These financial data are generally provided in a format that includes the following information:

Date
Open Price
High Price
Low Price
Closing Price
Volume
These data—often referred to as OHLC Chart Data—can be interpreted as Time Series data and are perfect for performing technical analysis. We’ll dive into this format in just a moment but, for now, just realize this is a standard format for historical pricing data within financial markets.

# yfinance: 
For this approach, we need to install the yfinance library as pip install yfinance. This library provides sample tools for working with financial data requests to the Yahoo Finance website. Keep in mind, however, this is not an official API and is subject to rate limiting, periodic breakage, and general quirkiness. Nonetheless, its the defacto Python library for OHLC data and can be used as follows:

In [ ]:
pip install yfinance

In [ ]:
import yfinance as yf
tick = ['NVDA','MSFT','AAC','AACG', 'AACI','AACIW','AADI','AAIC','AAL','AAMC','AAME','AAN','AAOI','AAON','AAP','AAPL','AAQC','AAT','AATC','AAU','AAWW','AB','ABB','ABBV','ABC']
for i in tick:
    data = yf.Ticker(i).history(period='10y')
    data.to_csv(i+".csv")

# Quandl:
Quandl offers official APIs to access any public dataset for free. Here we’ll see how to get OHLC data via the official Quandl python library and also via the pandas-datareader. One important note is that the free Quandl OHLC data only goes up to 2018 at the time of this article’s writing. If you need more recent data and don’t want to pay this source isn’t for you.

Quandl Python Library
To get started with Quadl’s official API we need to install the python library as such: pip install quandl. This will install the official quandl python library and let us make up to 50 daily API requests without registering an account. Let’s get our financial data:

In [ ]:
pip install quandl

In [ ]:
import quandl
import time
tick = ['NVDA','MSFT','AAC','AACG', 'AACI','AACIW','AADI','AAIC','AAL','AAMC','AAME','AAN','AAOI','AAON','AAP','AAPL','AAQC','AAT','AATC','AAU','AAWW','AB','ABB','ABBV','ABC']
# Get data via Quandl API
j=0
for i in tick:
    j+=1
    k=1
    if j>2*k+1:
        time.sleep(600)
        k+=20
    data = quandl.get('WIKI/NVDA')
    data.to_csv(i+".csv")



# Alpha Vantage:
The ** pandas-datareader library ** offers easy access to OHLC data via Alpha Vantage integration. The following code will retrieve historical data for different tickers once again:

In [ ]:
pip install alpha_vantage

In [ ]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
# Create an API object
ts = TimeSeries(key='UNO4CZQHSBZSN71N')
# Get daily OHLC data for NVDA
data, meta_data = ts.get_daily(symbol="NVDA")
data_df = pd.DataFrame(data)
data_df.to_csv("NVDA.csv", header=False)

## Alpha Vantage:
#### TIME_SERIES_DAILY_ADJUSTED Premium Trending

This API returns raw (as-traded) daily open/high/low/close/volume values, daily adjusted close values, and historical split/dividend events of the global equity specified, covering 20+ years of historical data.


API Parameters
❚ Required: function

The time series of your choice. In this case, function=TIME_SERIES_DAILY_ADJUSTED

❚ Required: symbol

The name of the equity of your choice. For example: symbol=IBM

❚ Optional: outputsize

By default, outputsize=compact. Strings compact and full are accepted with the following specifications: compact returns only the latest 100 data points; full returns the full-length time series of 20+ years of historical data. The "compact" option is recommended if you would like to reduce the data size of each API call.

❚ Optional: datatype

By default, datatype=json. Strings json and csv are accepted with the following specifications: json returns the daily time series in JSON format; csv returns the time series as a CSV (comma separated value) file.

❚ Required: apikey

Your API key


In [ ]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=IBM&apikey=1NSFEG3NTR9VZZOF'
r = requests.get(url)
data = r.json()

print(data)

## Zerodha-historical-data-download :
This contains code to download historical data for more than 2000 days and intraday data for more than 100 days. Download the framework and customize accordingly.

The changes you need to make.

Set your working directory where you want your data to be downloaded.

Enter your API key.

Enter youy API secret.

Generate request_token.

Set the Access token.

In the 'tickers' variable write all the stock names you want(write the names exactly as in zerodha app).

in the fetchOHLC func, enter the number of days.

Note: If you want F&O data, change the instruments('NSE') to instruments('NFO').

In [ ]:
!pip install kiteconnect --upgrade --quiet

In [ ]:
from kiteconnect import KiteConnect, KiteTicker
import pandas as pd
import datetime as dt
import numpy as np
import os
import time
import json



api_key = 'your_api_key'
api_secret = 'your_api_secret'

kite = KiteConnect(api_key)
url = kite.login_url()

request_token = 'your_request_token'
session = kite.generate_session(request_token, api_secret)
access_token = session['access_token']
kite.set_access_token(access_token)

instruments = pd.DataFrame(kite.instruments('NSE'))


def lookup(df, symbol):
    try:
        return df[df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1

def fetchandstoreOHLC(symbol,duration,tf):
    nodata=[]
    try:
        if not os.path.isfile(symbol+'.csv'):
            data = pd.DataFrame()
            instrument_token = lookup(instruments, symbol)  
            if duration>100:
                no_of_hundreds = duration//100
                start = 100
                end = 0
                for i in range(no_of_hundreds):
                    df = pd.DataFrame(kite.historical_data(instrument_token, from_date = dt.datetime.today()-dt.timedelta(start), 
                    to_date = dt.datetime.today()-dt.timedelta(end), interval = tf))
                    data = data.append(df)
                    start+=100
                    end+=100
                dur1 = duration%100
                df1 = pd.DataFrame(kite.historical_data(instrument_token, from_date = dt.datetime.today()-dt.timedelta(duration), 
                                      to_date = dt.datetime.today()-dt.timedelta(duration-dur1), interval = tf))
                data = data.append(df1)
            else:
                df2 = pd.DataFrame(kite.historical_data(instrument_token, from_date = dt.datetime.today()-dt.timedelta(duration), 
                                      to_date = dt.datetime.today(), interval = tf))
                data = data.append(df2)
            data.set_index("date", inplace = True)
            data.sort_index(ascending= True,inplace = True)
            print('Storing {} days data for {}'.format(duration, symbol))
            data.to_csv(symbol+'.csv')
        else:
            print('Data already available for{}'.format(symbol))
            return
    except:
        print("skipping for {}".format(symbol))
        nodata.append(symbol)
    
tickers = ['HDFCBANK','ICICIBANK','KOTAKBANK', 'AXISBANK', 'SBIN', 'RELIANCE','TCS','INFY','HINDUNILVR','HDFC','BAJFINANCE','WIPRO','BHARTIARTL','HCLTECH','ASIANPAINT','ITC','LT','ULTRACEMCO',
           'MARUTI','SUNPHARMA','TATASTEEL','JSWSTEEL','TITAN','ADANIPORTS','ONGC','HDFCLIFE','TECHM','DIVISLAB','POWERGRID','SBILIFE','NTPC','BAJAJ-AUTO','BPCL','IOC','M&M','SHREECEM','HINDALCO',
           'GRASIM','BRITANNIA','TATAMOTORS','COALINDIA','TATACONSUM','INDUSINDBK','DRREDDY','CIPLA','EICHERMOT','UPL','NESTLEIND','HEROMOTOCO','NIFTY 50','NIFTY BANK']


for ticker in tickers:
    fetchandstoreOHLC(ticker, 1095, '5minute')


# Angel One:
SMARTAPI-PYTHON is a Python library for dealing AMX,that is a set of REST-like HTTP APIs that expose many capabilities required to build stock market investment and trading platforms. It lets you execute orders in real time.
When a request is successfully placed the candle data for that time period is returned. The possible interval values can be referred from below table. API is as below

https://apiconnect.angelbroking.com/rest/secure/angelbroking/historical/v1/getCandleData
#### Exchange Constants
Param	Value	Description
exchange	NSE	NSE Equity
NFO	Only available for NSE Futures
#### Interval Constants
|Interval|	Description|
|---|---|
|ONE_MINUTE	|1 Minute|
|THREE_MINUTE |3 Minute|
|FIVE_MINUTE	|5 Minute|
|TEN_MINUTE	|10 Minute|
|FIFTEEN_MINUTE|15 Minute|
|THIRTY_MINUTE	|30 Minute|
|ONE_HOUR	|1 Hour|
|ONE_DAY	|1 Day|
#### Max Days in one Request
|Interval	|Max Days in one Request|
|---|---|
|ONE_MINUTE|	30|
|THREE_MINUTE|	90|
|FIVE_MINUTE	|90|
|TEN_MINUTE	|90|
|FIFTEEN_MINUTE|	180|
|THIRTY_MINUTE	|180|
|ONE_HOUR	|365|
|ONE_DAY	|2000|
### Get Candle Data
All requests and its response structure is as below.  
Get Candle Data Request  
{  
     "exchange": "NSE",  
     "symboltoken": "3045",  
     "interval": "ONE_MINUTE",  
     "fromdate": "2021-02-10 09:15",  
     "todate": "2021-02-10 09:16"  
}  
#### Get Candle Data Response  
{  
     "status": true,  
     "message": "SUCCESS",  
     "errorcode": "",  
     "data": [  
          [  
               "2021-02-10T09:15:00+05:30",  
               394.05,  
               397.7,  
               394,  
               396.3,  
               722616  
          ],  
          [  
               "2021-02-10T09:16:00+05:30",  
               396.25,  
               396.65,  
               395.85,  
               395.85,  
               391702  
          ]  
     ]  
}  
  
NOTE:  
Historical API only available for NSE Equity(NSE) and NSE Futures (NFO) Segment  
In Get Candle Data Request fromdate and todate format should be "yyyy-MM-dd hh:mm"  
The response is an array of records, where each record in turn is an array of the following values — [timestamp, open, high, low, close, volume].  

In [ ]:

pip install websocket

In [ ]:
from smartapi.smartConnect import SmartConnect
import pandas as pd
import requests
feed_token = None
token_map = None

obj = SmartConnect(api_key="LreBghDE")
data = obj.generateSession("123456", "Tannu@123")
print(data)
refreshToken=data['data']['refreshToken']

#fetch the feedtoken
feedToken = obj.getfeedToken()
feed_token = feedToken
#fetch User Profile
userProfile = obj.getProfile((refreshToken))
print(userProfile)

#historic api
def OHLCHistory(symbol, token, interval, fdate, todate):
    try:
        historicParam={
            "exchange":"NSE",
            "tradingsymbol":symbol,
            "symboltoken":token,
            "interval":interval,
            "fromdate":fdate,
            "todate":todate
        }
        history = obj.getCandleData(historicParam)['data']
        history = pd.DataFrame(history)

        history = history.rename(
            columns={0: "DateTime", 1: "open", 2: "high", 3: "low", 4: "close", 5:"Volumne"}
        )
        history['DateTime']=pd.to_datetime(history['DataTime'])
        history=history.set_index('DateTime')
        return history
    except Exception as e:
        print("Histpric api failed: {}".format(e))
data = OHLCHistory("SBIN-EQ", "3045", "ONE_DAY", "2020-02-08 00:00", "2021-02-08 15:30")
data_df = pd.DataFrame(data)
data_df.to_csv(r'{}.csv'.format("SBIN_EQ"), index=True, header=False)

# Polygio.io

## Parameters
stocksTicker
*
AAPL
The ticker symbol of the stock/equity.

multiplier
*
1
The size of the timespan multiplier.

timespan
*

day
The size of the time window.

from
*
2021-07-22
The start of the aggregate time window. Either a date with the format YYYY-MM-DD or a millisecond timestamp.

to
*
2021-07-22
The end of the aggregate time window. Either a date with the format YYYY-MM-DD or a millisecond timestamp.

adjusted

Whether or not the results are adjusted for splits. By default, results are adjusted. Set this to false to get results that are NOT adjusted for splits.

sort

asc
Sort the results by timestamp. asc will return results in ascending order (oldest at the top), desc will return results in descending order (newest at the top).

limit
120
Limits the number of base aggregates queried to create the aggregate results. Max 50000 and Default 5000. Read more about how limit is used to calculate aggregate results in our article on Aggregate Data API Improvements.

In [ ]:
pip install polygon-api-client
pip install local_settings

In [ ]:
from sqlite3 import adapters
from polygon import RESTClient
from local_settings import polygon as settings
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import pandas as pd
import datetime

markets = ['crypto', 'stocks']

class MyRESTClient(RESTClient):
    def __init__(self, auth_key: str, timeout: int=None):
        super().__init__(auth_key)
        retry_strategy = Retry(total=10, backoff_factor=10, status_forcelist=[429, 500, 503, 504])
        adapter = HTTPAdapter(max_retries=retry_strategy)
        self._session.mount('https://', adapter)
    
    def get_bars(self, market:str=None, ticker:str=None, multiplier:int=1, timespan:str='minute', from_:date=None, to:date=None) -> pd.DataFrame:
        if not market in markets:
            raise Exception(f'Market must be one of {markets}.')
        
        if ticker is None:
            raise Exception('Ticker must not be None.')
        
        from_ = from_ if from_ else data(2000, 1, 1)
        to = to if to else date.today()

        if market == 'stocks':
            resp = self.stocks_aggregates(ticker, multiplier, timespan, from_.strftime('%Y-%m-%d'), to..strftime('%Y-%m-%d'), limit=50000)
            df = pd.DataFrame(resp.results)
            last_minute = 0
            while resp.results[-1]['t'] > last_minute:
                last_minute = resp.results[-1]['t']
                last_minute_date = datetime.fromtimestamp(last_minute/1000).strftime('%Y-%m-%d')
                resp = self.stocks_aggregates(ticker, multiplier, timespan, from_.strftime('%Y-%m-%d'), to..strftime('%Y-%m-%d'), limit=50000)
                new_bars = pd.DataFrame(resp.results)
                df = df.append(new_bars[new_bars['t']>last_minute])

            df['date'] = pd.to_datetime(df['t'], unit='ms')
            df = df.rename(columns={
                'o':'open',
                'h':'high',
                'l':"low",
                'c':'close',
                'v':'volume',
                'vw':'vwap',
                'n':'transactions'})
            
            df = df[['date', 'open', 'high', 'low' ,'close', 'volume']]
            return df
    return None


client = MyRESTClient(settings['api_key'])
ticker='AAPL'
df = client.get_bars(market="stocks", ticker=ticker)
df.to_csv(r'{}.csv'.format(ticker), header=False)

# Kotak Securities

All URIs are relative to https://tradeapi.kotaksecurities.com/apim

|Class|	Method|	Description|
|---|---|---|
|SessionApi	|ks_api.KSTradeApi	|Initialise Session
|SessionApi	|login	|Login using Userid
|SessionApi	|session_2fa|	Generate final Session Token
|OrderApi	|place_order|	Place a New order
|OrderApi	|modify_order|	Modify an existing order
|OrderApi	|cancel_order|	Cancel an order
|ReportsApi	|order_report|	Get order report
|ReportsApi	|trade_report|	Get trade report
|MarginApi	|margin_required|	Get Margin Required for an order by amount or quantity.
|PositionsApi	|positions	|Get's Open position.
|QuoteApi	|quote	|Get Quote details
|HistoricalApi|	history|	Get historical data.
|SessionApi	|logout	|Invalidate Session Token

#### get_resource
object history(resource, input)  
Get historical data based on given resource  
Get Historical data  

#### Parameters
|Name|	Type|	Description	Notes|
|---|---|---|
|resource|	str	|	Type of resource historicalprices,historicalprices-unadjusted,NSEFNO_HistoricalContinuousChart,LiveorEODHistorical|
|input	|str	|	Json as per resource selected|

#### Return type
object

#### HTTP request headers
Accept: application/json

#### HTTP response details
|Status code	|Description|
|---|---|
|200|	Historical Details|	

In [ ]:
pip install git+https://github.com/osparamatrix/ks-orderapi-python.git

In [ ]:
from ks_api_client import ks_api

client = ks_api.KSTradeApi(access_token = "access_token", userid = "userid", \
                 consumer_key = "consumer_key", ip = "IP", app_id = "app_id")

#First initialize session and generate session token

try:
    # Get historical prices
    client.history("historicalprices",{"exchange":"bse","cocode":"476","fromdate":"01-jan-2014","todate":"08-oct-2015"})
except Exception as e:
    print("Exception when calling Historical API->details: %s\n" % e)

# ALice Blue API

Alice Blue Python library provides an easy to use wrapper over the HTTPS APIs.  
The HTTP calls have been converted to methods and JSON responses are wrapped into Python-compatible objects.  
Websocket connections are handled automatically within the library.  

#### Installation
To force upgrade existing installations:  

pip uninstall alice_blue  
pip --no-cache-dir install --upgrade alice_blue  

#### Getting Started with API

There is only one class in the whole library: AliceBlue. The login_and_get_sessioID() static method is used to retrieve session ID from alice blue server. A session ID is valid for 24 hours. With session ID, you can instantiate an AliceBlue object. Ideally you only need to create a session ID once every day. Once the session ID is created new, it'll be stored in a temporary location. Next time, the same session ID will be used.  

#### Logging
The whole library is equipped with python's logging module for debugging. If more debug information is needed, enable logging using the following code.  
import logging  
logging.basicConfig(level=logging.DEBUG)  

#### Getting an session ID
1. Import alice_blue  
2. Create session_id using login_and_get_sessioID() function with your username, password, 2FA (2fa is now year of birth), app_id and api_secret.  

#### Problem getting access token
If you are facing problem getting access token, make sure the following are correct.  
username.  
password.  
2FA.  
api secret.  
app id.  
Even after verifying all these, if you are facing problem, contact alice customer care. They should enable the API access in their end.  

#### Historical API
Alice Blue now supports downloading historical data for back testing.  
  
1. Historical data API will be available from 5:30 PM (evening) to 8 AM (Next day morning) on weekdays (Monday to Friday). Historical data will not be available during market hours.  
2. Historical data API will be available fully during weekends and holidays.  
3. For NSE segment, 2 years of historical data will be provided.  
4. For NFO, CDS and MCX segments, current expiry data will be provided.  

In [ ]:
pip --no-cache-dir install --upgrade alice_blue

In [ ]:
from alice_blue import *
session_id = AliceBlue.login_and_get_sessioID(username = "username", password = "password", twoFA= "1993", app_id = "app_id", api_secret = "api_secret")
data = pd.DataFrame(alice.historical_data(alice.get_instrument_by_symbol("NSE"), datetime.datetime.now(), datetime.datetime.now(), HistoricalDataType.Minute))
data.to_csv("NSE.csv", header=False)